In [57]:
from numpy import *
from scipy.sparse import *
from pandas import *

In [8]:
tracksWithTags=load("tracks_with_tags.npy")
tracksID=tracksWithTags[:,0]
tracksTags=tracksWithTags[:,1:]

In [10]:
tracksID.shape,tracksTags.shape

((100000,), (100000, 5))

In [17]:
with open("two_colums_tracks_tags.txt","w") as f:
    for trackIDandTags in tracksWithTags:
        trackID=str(trackIDandTags[0])
        for tag in trackIDandTags[1:]:
            f.write(trackID + " " + str(tag) + "\n")

In [20]:
genfromtxt("two_colums_tracks_tags.txt",dtype=int32)[0:10]

array([[2972914,   54087],
       [2972914,    1757],
       [2972914,    1718],
       [2972914,  116712],
       [2972914,  189631],
       [2750239,  189631],
       [2750239,    3424],
       [2750239,  177424],
       [2750239,   46208],
       [2750239,  205245]], dtype=int32)

In [64]:
data=read_table("two_colums_tracks_tags.txt",
                sep=" ",
                usecols=[0,1],
                names=["track","tag"])

In [68]:
#data

In [66]:
data["track"]=data["track"].astype("category")
data["tag"]=data["tag"].astype("category")

In [51]:
matrixTracksTags=csr_matrix((ones(500000),
                            (data["track"].cat.codes.copy(),
                             data["tag"].cat.codes.copy())))
matrixTracksTags

<100000x31901 sparse matrix of type '<class 'numpy.float64'>'
	with 487328 stored elements in Compressed Sparse Row format>

In [52]:
matrixTracksTags=matrixTracksTags[:,1:]
matrixTracksTags=csc_matrix(matrixTracksTags)#tolta la colonna che contava il tag 0

In [53]:
matrixTracksTags

<100000x31900 sparse matrix of type '<class 'numpy.float64'>'
	with 483482 stored elements in Compressed Sparse Column format>

In [58]:
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)

In [73]:
MODELS = {"tfidf": TFIDFRecommender,
          "cosine": CosineRecommender}

In [74]:
def get_model(model_name):
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    params = {}

    return model_class(**params)

In [149]:
modelTF=get_model("tfidf")

In [150]:
modelTF.fit(matrixTracksTags)

In [151]:
simTF=(modelTF.similarity)
simTF.setdiag(0)
simTF

/home/oppy/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


<100000x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 1945658 stored elements in Compressed Sparse Row format>

In [286]:
targetIDs=unique(genfromtxt("target_tracks.csv",skip_header=1,dtype=int32))
allTracks=array(list(data["track"].cat.categories))
targetIDs.size,allTracks.size

(32195, 100000)

In [189]:
myMask=list(map(lambda x: where(targetIDs==x)[0].size>0,allTracks))

In [305]:
simTarget=simTF[array(myMask)]
simTarget=simTarget.T
simTarget

<100000x32195 sparse matrix of type '<class 'numpy.float64'>'
	with 597938 stored elements in Compressed Sparse Column format>

In [351]:
targetIDs.sort()
targetIDs

array([    252,     874,    1416, ..., 5007354, 5014702, 5018274])

In [233]:
allPlTracks=load_npz("all_playlists_with_tracks.npz")[:,0]
allPlOnlyTracks=load_npz("all_playlists_with_tracks.npz")[:,1:]
allPlTracks=allPlTracks.data
allPlTracks

array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362])

In [224]:
targetPl=genfromtxt("target_playlists.csv",skip_header=1,dtype=int32)
targetPl.sort()
targetPl

array([    7614,     7692,     7816, ..., 11765404, 11765405, 11766362], dtype=int32)

In [247]:
monoArtPlIDs=load("target_playlist_mono_artist.npy")[:,0]
monoArtPlArt=load("target_playlist_mono_artist.npy")[:,1]

In [275]:
artTracks=load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")[:,1:]

In [274]:
uniqueArt=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")

In [420]:
"""
64742
5895355
6231219
6264193
6341449
6779046
7418280
"""
with open("veryNewSim.csv","a") as f:
    for plID,plTracks in zip(allPlTracks,allPlOnlyTracks):
        if(where(targetPl==plID)[0].size>0 and plID>=7974833):# la playlist è target
            f.write(str(plID)+",")
            thisTracks=plTracks.data
            isMonoPL=where(monoArtPlIDs==plID)[0]
            if(isMonoPL.size>0):#la pl è monoartist
                artist=monoArtPlArt[isMonoPL[0]]
                artIndex=where(uniqueArt==artist)[0][0]
                toSuggestWithDuplicates=artTracks[artIndex].data
                #print(thisArtTracks)
                #break
            else:
                trackSims=[]
                candidateIndexes=[]
                for track in thisTracks:
                    index=where(allTracks==track)
                    trackSims+=list(simTarget[index].data)
                    candidateIndexes+=list(simTarget[index].nonzero()[1])
                topSimilarityIndex=flip(argsort(trackSims),0)
                if(topSimilarityIndex.size!=0):
                    mostSimilarTrackIndex=array(candidateIndexes)[topSimilarityIndex]
                    toSuggestWithDuplicates=targetIDs[mostSimilarTrackIndex]
                #print(toSuggestWithDuplicates.size)
            if(topSimilarityIndex.size!=0):
                noDuplicates=array([t for t in toSuggestWithDuplicates if t not in thisTracks])
                #print(noDuplicates.size)
                final=noDuplicates.take(range(min(5,noDuplicates.size)))
            else:
                final=array([])
            for t in final:
                f.write(str(t)+" ")
            if(final.size<5):
                print(plID)
            f.write("\n")

7974833
8036589
8039237
8469697
10179818
11319088
11542825
